### Group Project 4 : Comparing 3 Models for Predicting Recidivism

For background on this project, please see the [README](../README.md).

**Notebooks**
- Data Acquisition & Cleaning (this notebook)
- [Exploratory Data Analysis](./02_eda.ipynb)
- [Modeling](./03_modeling.ipynb)
- [Results and Recommendations](./04_results.ipynb)

**In this notebook, you'll find (for each of the 3 models):**
- Data ingestion
- Cleaning
- New feature engineering
- etc. TODO

In [103]:
import pandas as pd

**Model 1: Base feature set - New York**

TODO provide some background

**Model 2: Criminal history feature set - Florida**

The Florida dataset comprises a number of SQLite tables related to criminal history for around 11,000 Broward County citizens. This data was collected as part of the evaluation of the Correctional Offender Management Profiling for Alternative Sanctions (COMPAS) decision support tool used in Broward County and in other U.S. states - further descriptions available on [Wikipedia](https://en.wikipedia.org/wiki/COMPAS_(software)) - so there are some COMPAS scores within this dataset as well.

In order to retrieve the data and put it into a form that was conducive to Pandas analysis, two SQL queries were constructed:

- [people.sql](../data/FL/people.sql) - basic demographic data, incarceration dates, and COMPAS scores
- [charges.sql](../data/FL/charges.sql) - case management, charge and arrest data

The results of those queries were then exported to CSV as [final_people.csv](../data/FL/final_people.csv) and [final_charges.csv](../data/fl/final_charges.csv) within the **data** subfolder, and will serve as the basis for the rest of this cleaning and transformation exercise.

In [104]:
# read in people dataset and take a look
people = pd.read_csv('../data/FL/final_people.csv')
people.head()

,person_id,sex,race,birth_date,first_incarceration_date,first_incarceration_release,last_incarceration_date,last_incarceration_release,num_incarcerations,comp_f_min_score,...,comp_f_max_score,comp_f_max_decile,comp_r_min_score,comp_r_min_decile,comp_r_max_score,comp_r_max_decile,comp_v_min_score,comp_v_min_decile,comp_v_max_score,comp_v_max_decile
0,1,Male,Other,1947-04-18 00:00:00.000000,2013-08-13 06:03:42.000000,2013-08-14 05:41:20.000000,2014-07-07 09:26:12.000000,2014-07-14 08:24:15.000000,2,13,...,13,1,-2.78,1,-2.78,1,-4.31,1,-4.31,1
1,2,Male,Caucasian,1985-02-06 00:00:00.000000,2014-12-30 10:47:52.000000,2015-01-03 02:18:24.000000,2014-12-30 10:47:52.000000,2015-01-03 02:18:24.000000,1,16,...,16,2,-0.34,5,-0.34,5,-2.75,2,-2.75,2
2,3,Male,African-American,1982-01-22 00:00:00.000000,2015-10-15 00:00:00.000000,2015-12-07 00:00:00.000000,2013-01-26 03:45:27.000000,2013-02-05 05:36:53.000000,2,25,...,25,6,-0.76,3,-0.76,3,-3.07,1,-3.07,1
3,4,Male,African-American,1991-05-14 00:00:00.000000,2013-04-13 04:58:34.000000,2013-04-14 07:02:04.000000,2016-01-08 09:59:55.000000,2016-01-09 04:41:39.000000,5,26,...,26,7,-0.66,4,-0.66,4,-2.26,3,-2.26,3
4,5,Male,African-American,1993-01-21 00:00:00.000000,NaN,NaN,NaN,NaN,0,19,...,19,3,0.16,8,0.16,8,-1.59,6,-1.59,6


In [105]:
# check people data types
people.dtypes

person_id                        int64
sex                             object
race                            object
birth_date                      object
first_incarceration_date        object
first_incarceration_release     object
last_incarceration_date         object
last_incarceration_release      object
num_incarcerations               int64
comp_f_min_score                 int64
comp_f_min_decile                int64
comp_f_max_score                 int64
comp_f_max_decile                int64
comp_r_min_score               float64
comp_r_min_decile                int64
comp_r_max_score               float64
comp_r_max_decile                int64
comp_v_min_score               float64
comp_v_min_decile                int64
comp_v_max_score               float64
comp_v_max_decile                int64
dtype: object

In [106]:
# convert all datetime columns
people['birth_date'] = pd.to_datetime(people['birth_date'])
people['first_incarceration_date'] = pd.to_datetime(people['first_incarceration_date'])
people['first_incarceration_release'] = pd.to_datetime(people['first_incarceration_release'])
people['last_incarceration_date'] = pd.to_datetime(people['last_incarceration_date'])
people['last_incarceration_release'] = pd.to_datetime(people['last_incarceration_release'])

In [107]:
# dummify sex and race
people = pd.get_dummies(data = people, columns = ['sex', 'race'], drop_first = True)

In [108]:
# final dtype/null check
people.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 11757 entries, 0 to 11756
Data columns (total 25 columns):
 #   Column                       Non-Null Count  Dtype         
---  ------                       --------------  -----         
 0   person_id                    11757 non-null  int64         
 1   birth_date                   11757 non-null  datetime64[ns]
 2   first_incarceration_date     11127 non-null  datetime64[ns]
 3   first_incarceration_release  11127 non-null  datetime64[ns]
 4   last_incarceration_date      11127 non-null  datetime64[ns]
 5   last_incarceration_release   11127 non-null  datetime64[ns]
 6   num_incarcerations           11757 non-null  int64         
 7   comp_f_min_score             11757 non-null  int64         
 8   comp_f_min_decile            11757 non-null  int64         
 9   comp_f_max_score             11757 non-null  int64         
 10  comp_f_max_decile            11757 non-null  int64         
 11  comp_r_min_score             11757 non-nu

CONCLUSIONS
- Our data types are now looking good
- It appears that there are 630 people who have no incarceration data (the nulls from above) - they should be dropped from our analysis, since at least an initial incarceration is required to qualify for recidivism
- The COMPAS scores could be interesting for later comparison, but will not be used as features, so they can be saved to a separate file for now

In [109]:
people = people[people['num_incarcerations'] > 0]

In [110]:
compas = people[['person_id'] + [col for col in people if col.startswith('comp_')]]
compas.to_csv('../data/FL/final_compas.csv')
people = people.drop(columns = [col for col in people if col.startswith('comp_')])
people.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 11127 entries, 0 to 11756
Data columns (total 13 columns):
 #   Column                       Non-Null Count  Dtype         
---  ------                       --------------  -----         
 0   person_id                    11127 non-null  int64         
 1   birth_date                   11127 non-null  datetime64[ns]
 2   first_incarceration_date     11127 non-null  datetime64[ns]
 3   first_incarceration_release  11127 non-null  datetime64[ns]
 4   last_incarceration_date      11127 non-null  datetime64[ns]
 5   last_incarceration_release   11127 non-null  datetime64[ns]
 6   num_incarcerations           11127 non-null  int64         
 7   sex_Male                     11127 non-null  uint8         
 8   race_Asian                   11127 non-null  uint8         
 9   race_Caucasian               11127 non-null  uint8         
 10  race_Hispanic                11127 non-null  uint8         
 11  race_Native American         11127 non-nu

CONCLUSIONS
- Our demographic (birth date/sex/race) and incarceration statistics are now in appropriate format for feature engineering
- Now we need to incorporate data related to arrests and charges

In [111]:
# read in charges dataset and take a look
charges = pd.read_csv('../data/FL/final_charges.csv')
charges.head()

,person_id,case_number,offense_date,charge_degree,charge,arrest_date
0,1,09083797TI30A,2009-08-11 00:00:00.000000,(0),Unlawful Speed (Requires Speeds),NaN
1,1,09098832TI20A,2009-10-24 00:00:00.000000,(0),Speed/65 Interstate,NaN
2,1,13009443TI30A,2013-01-14 00:00:00.000000,(0),Disobey/Ran Stop Sign,NaN
3,1,13009443TI30A,2013-01-14 00:00:00.000000,(0),Expired Tag/Infraction,NaN
4,1,13011352CF10A,2013-08-13 00:00:00.000000,(F3),Aggravated Assault w/Firearm,2013-08-13 00:00:00.000000


In [112]:
charges.dtypes

person_id         int64
case_number      object
offense_date     object
charge_degree    object
charge           object
arrest_date      object
dtype: object

In [113]:
# convert datetimes
charges['offense_date'] = pd.to_datetime(charges['offense_date'])
charges['arrest_date'] = pd.to_datetime(charges['arrest_date'])

In [114]:
# final dtypes/null check
charges.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 140034 entries, 0 to 140033
Data columns (total 6 columns):
 #   Column         Non-Null Count   Dtype         
---  ------         --------------   -----         
 0   person_id      140034 non-null  int64         
 1   case_number    140034 non-null  object        
 2   offense_date   140034 non-null  datetime64[ns]
 3   charge_degree  140034 non-null  object        
 4   charge         139605 non-null  object        
 5   arrest_date    92150 non-null   datetime64[ns]
dtypes: datetime64[ns](2), int64(1), object(3)
memory usage: 6.4+ MB


CONCLUSIONS
- We will collect first/last dates and charge degrees for charges and arrests per person as potentially useful features
- We will retrieve counts of charges by degree per person (pivoted) as well
- We will collect aggregated charge text data for NLP
- We will calculate mean time between offenses and between arrests

In [115]:
# Collect first/last charge and arrest data
# Using the offense date as our "first/last" marker
# And add to people dataframe
people['first_arrest_date'] = \
    people.join(charges.sort_values(by = ['person_id', 'offense_date']).groupby('person_id')['arrest_date'].first(),
    on = 'person_id',
    how = 'left')['arrest_date']
people['last_arrest_date'] = \
    people.join(charges.sort_values(by = ['person_id', 'offense_date']).groupby('person_id')['arrest_date'].last(),
    on = 'person_id',
    how = 'left')['arrest_date']
people['first_charge_degree'] = \
    people.join(charges.sort_values(by = ['person_id', 'offense_date']).groupby('person_id')['charge_degree'].first(),
    on = 'person_id',
    how = 'left')['charge_degree']
people['last_charge_degree'] = \
    people.join(charges.sort_values(by = ['person_id', 'offense_date']).groupby('person_id')['charge_degree'].last(),
    on = 'person_id',
    how = 'left')['charge_degree']

In [116]:
# pivot the degrees of charges into counts per person
# and add to people dataframe
charge_pivot = pd.pivot_table(charges, index = 'person_id', columns = 'charge_degree', aggfunc = 'count')['arrest_date']
charge_pivot.columns = ['charge_degree_count_' + col.replace('(', '').replace(')', '') for col in charge_pivot.columns]
people = people.join(charge_pivot, on = 'person_id', how = 'left')


In [117]:
# aggregate charges per person for NLP
# and add to people dataframe
# https://stackoverflow.com/questions/27298178/concatenate-strings-from-several-rows-using-pandas-groupby
agg_charges = charges[charges['charge'].notnull()].groupby('person_id').agg({'charge': '|'.join})
people['agg_charges'] = \
    people.join(agg_charges, on = 'person_id', how = 'left')['charge']

In [118]:
# TODO - maybe EDA, but don't use these for model!!
# calculate mean time between offenses and between arrests per person
# and add to people dataframe
# https://stackoverflow.com/questions/45241221/python-pandas-calculate-average-days-between-dates
charges['previous_offense'] = \
    charges.sort_values(by = ['person_id', 'offense_date']).groupby(['person_id'])['offense_date'].shift(1)
charges['days_between_offenses'] = \
    (charges['offense_date'] - charges['previous_offense']).apply(lambda x: x.days)
charges['previous_arrest'] = \
    charges.sort_values(by = ['person_id', 'arrest_date']).groupby(['person_id'])['arrest_date'].shift(1)
charges['days_between_arrests'] = \
    (charges['arrest_date'] - charges['previous_arrest']).apply(lambda x: x.days)

# need to remove zeros - don't want them affecting the averages
days_between_offenses = charges[charges['days_between_offenses'] > 0].groupby('person_id')['days_between_offenses'].mean()
days_between_arrests = charges[charges['days_between_arrests'] > 0].groupby('person_id')['days_between_arrests'].mean()

people['avg_days_between_offenses'] = \
    people.join(days_between_offenses, on = 'person_id', how = 'left')['days_between_offenses']
people['avg_days_between_arrests'] = \
    people.join(days_between_arrests, on = 'person_id', how = 'left')['days_between_arrests']

In [119]:
people.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 11127 entries, 0 to 11756
Data columns (total 38 columns):
 #   Column                          Non-Null Count  Dtype         
---  ------                          --------------  -----         
 0   person_id                       11127 non-null  int64         
 1   birth_date                      11127 non-null  datetime64[ns]
 2   first_incarceration_date        11127 non-null  datetime64[ns]
 3   first_incarceration_release     11127 non-null  datetime64[ns]
 4   last_incarceration_date         11127 non-null  datetime64[ns]
 5   last_incarceration_release      11127 non-null  datetime64[ns]
 6   num_incarcerations              11127 non-null  int64         
 7   sex_Male                        11127 non-null  uint8         
 8   race_Asian                      11127 non-null  uint8         
 9   race_Caucasian                  11127 non-null  uint8         
 10  race_Hispanic                   11127 non-null  uint8         
 11  ra

CONCLUSIONS
- It appears that there are about 500 people with no arrest data - they should be dropped from our analysis
- The **first_charge_degree** and **last_charge_degree** columns should be dummified
- Zeros should be imputed for the **charge_degree_XXX** columns
- Empty string should be imputed for **agg_charges** column
- For people with nulls in the **avg_days_between_XXX** columns, who are probably non-recidivists, it makes sense to impute a very large number

In [120]:
people = people[people['first_arrest_date'].notnull()]
people.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 10645 entries, 0 to 11756
Data columns (total 38 columns):
 #   Column                          Non-Null Count  Dtype         
---  ------                          --------------  -----         
 0   person_id                       10645 non-null  int64         
 1   birth_date                      10645 non-null  datetime64[ns]
 2   first_incarceration_date        10645 non-null  datetime64[ns]
 3   first_incarceration_release     10645 non-null  datetime64[ns]
 4   last_incarceration_date         10645 non-null  datetime64[ns]
 5   last_incarceration_release      10645 non-null  datetime64[ns]
 6   num_incarcerations              10645 non-null  int64         
 7   sex_Male                        10645 non-null  uint8         
 8   race_Asian                      10645 non-null  uint8         
 9   race_Caucasian                  10645 non-null  uint8         
 10  race_Hispanic                   10645 non-null  uint8         
 11  ra

In [121]:
people['first_charge_degree'] = people['first_charge_degree'].str.replace('(', '', regex = False).str.replace(')', '', regex = False)
people['last_charge_degree'] = people['last_charge_degree'].str.replace('(', '', regex = False).str.replace(')', '', regex = False)
people = pd.get_dummies(data = people, columns = ['first_charge_degree', 'last_charge_degree'], drop_first = True)

In [122]:
people[[col for col in people if col.startswith('charge_degree_')]] = people[[col for col in people if col.startswith('charge_degree_')]].fillna(value = 0)
people.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 10645 entries, 0 to 11756
Data columns (total 63 columns):
 #   Column                          Non-Null Count  Dtype         
---  ------                          --------------  -----         
 0   person_id                       10645 non-null  int64         
 1   birth_date                      10645 non-null  datetime64[ns]
 2   first_incarceration_date        10645 non-null  datetime64[ns]
 3   first_incarceration_release     10645 non-null  datetime64[ns]
 4   last_incarceration_date         10645 non-null  datetime64[ns]
 5   last_incarceration_release      10645 non-null  datetime64[ns]
 6   num_incarcerations              10645 non-null  int64         
 7   sex_Male                        10645 non-null  uint8         
 8   race_Asian                      10645 non-null  uint8         
 9   race_Caucasian                  10645 non-null  uint8         
 10  race_Hispanic                   10645 non-null  uint8         
 11  ra

In [123]:
people[['agg_charges']] = people[['agg_charges']].fillna(value = '')
people.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 10645 entries, 0 to 11756
Data columns (total 63 columns):
 #   Column                          Non-Null Count  Dtype         
---  ------                          --------------  -----         
 0   person_id                       10645 non-null  int64         
 1   birth_date                      10645 non-null  datetime64[ns]
 2   first_incarceration_date        10645 non-null  datetime64[ns]
 3   first_incarceration_release     10645 non-null  datetime64[ns]
 4   last_incarceration_date         10645 non-null  datetime64[ns]
 5   last_incarceration_release      10645 non-null  datetime64[ns]
 6   num_incarcerations              10645 non-null  int64         
 7   sex_Male                        10645 non-null  uint8         
 8   race_Asian                      10645 non-null  uint8         
 9   race_Caucasian                  10645 non-null  uint8         
 10  race_Hispanic                   10645 non-null  uint8         
 11  ra

In [124]:
people[['avg_days_between_offenses']] = people[['avg_days_between_offenses']].fillna(value = 999999)
people[['avg_days_between_arrests']] = people[['avg_days_between_arrests']].fillna(value = 999999)
people.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 10645 entries, 0 to 11756
Data columns (total 63 columns):
 #   Column                          Non-Null Count  Dtype         
---  ------                          --------------  -----         
 0   person_id                       10645 non-null  int64         
 1   birth_date                      10645 non-null  datetime64[ns]
 2   first_incarceration_date        10645 non-null  datetime64[ns]
 3   first_incarceration_release     10645 non-null  datetime64[ns]
 4   last_incarceration_date         10645 non-null  datetime64[ns]
 5   last_incarceration_release      10645 non-null  datetime64[ns]
 6   num_incarcerations              10645 non-null  int64         
 7   sex_Male                        10645 non-null  uint8         
 8   race_Asian                      10645 non-null  uint8         
 9   race_Caucasian                  10645 non-null  uint8         
 10  race_Hispanic                   10645 non-null  uint8         
 11  ra

CONCLUSIONS
- We've resolved all null issues
- We lost about 1,000 rows, but the vast majority of those were people for whom recidivism was incalculable, so it's an acceptable loss
- We will keep agg_charges to the side for potential NLP analysis in vectorized form later
- We have 7 datetime columns that need to be reengineered into numerics, since we're not attempting time series analysis here - the most useful scenario is probably to convert the non-date-of-birth columns into "years since birth" quantities by referencing the date of birth column
- There are also some potentially interesting features we could engineer that summarize charges - total number of charges, number of felonies/misdemeanors/infractions

In [ ]:
# We'll use this to determine folks with no charges to drop, and potentially also useful as a feature
people['total_charge_count'] = people[[col for col in people if col.startswith('charge_degree_count_')]].sum(axis = 1)

**Model 3: Behavioral feature set - Georgia**

TODO provide some background

**FINAL NOTES**:
- The final datasets for modeling are exported:
  - [here](../data/NY/NY_final.csv) for Model 1 (NY)
  - [here](../data/FL/FL_final.csv) for Model 2 (FL)
  - [here](../data/GA/GA_final.csv) for Model 3 (GA)
- The next notebook in the series is [Exploratory Data Analysis](./02_eda.ipynb).